# MQTT

MQTT Discovery and updates for homeassistant.

`/home/mosquitto/config/mosquitto.conf`

```yaml
allow_anonymous true
listener 1883 0.0.0.0
```

## Broker

In [ ]:
import asyncio
import aiomqtt

URL = "mqtt"

async def counter(client: aiomqtt.Client):
    for i in range(3):
        print(f"Publishing {i}")
        await client.publish("counter/count", payload=f'value={i}')
        await asyncio.sleep(0.1)

async def subscriber(client: aiomqtt.Client):
    await client.subscribe("#")
    async for message in client.messages:
        print(f"GOT {message.topic}: {message.payload}")

print("connecting ...")
async with aiomqtt.Client(URL) as client:
    print("client", client)
    try:
        asyncio.create_task(subscriber(client))
        await counter(client)
        # await asyncio.sleep(600)
    except asyncio.CancelledError:
        pass

print("done")


## Home Assistand Discovery Example

In [ ]:
import asyncio
import json
import aiomqtt

# https://www.home-assistant.io/integrations/sensor/#device-class

msg1 = {
  "name": "Temperature",
  "state_topic": "leaf/state/node-freezer-temperature",
  "device_class": "temperature",  # sets icon
  "unique_id": "node-freezer-temperature",
  "object_id": "node-freezer-temperature",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

msg2 = {
  "name": "Humidity",
  "state_topic": "leaf/state/node-freezer-humidity",
  "device_class": "humidity",  # sets icon
  "unique_id": "node-freezer-humidity",
  "object_id": "node-freezer-humidity",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

async with aiomqtt.Client("mqtt") as client:
    # discover
    await client.publish("homeassistant/sensor/node-freezer-temperature/config", payload=json.dumps(msg1))
    await client.publish("homeassistant/sensor/node-freezer-humidity/config", payload=json.dumps(msg2))
    # update
    for i in range(10):
        await client.publish("leaf/state/node-freezer-temperature", payload=i)
        await client.publish("leaf/state/node-freezer-humidity", payload=-i)
        await asyncio.sleep(1)
    # remove
    await client.publish("homeassistant/sensor/node-freezer-temperature/config")
    await client.publish("homeassistant/sensor/node-freezer-humidity/config")

## Eventbus / devices

In [1]:
import asyncio
import aiomqtt
import json
from eventbus import bus, Device, Light
from eventbus.devices import UID_SEP

@bus.on("*")
def printer(**event):
    pass
    # print("GOT", json.dumps(event, indent=2))
    print("GOT", event)


async def cb(device: Device, uid, value, **rest):
    # set light to requested state ..., then:
    print("CB", uid, value)
    await device.update(uid, value)

light1 = Light("kitchen_light", cb)


URL = "mqtt"

class HassMqtt:

    def __init__(self, client):
        self.client = client
        asyncio.create_task(self.action_listener())

        @bus.on("!device")
        async def discover(uid, domain, attributes, entities, **rest):
            # homeassistant unique_id (no periods)
            unique_id = uid.replace(UID_SEP, "-")
            msg = attributes.copy()
            msg["unique_id"] = msg["object_id"] = unique_id
            if "name" not in msg: msg["name"] = msg["name"] = uid.split(UID_SEP)[1].replace("_", " ").title()
            for uid, entity in entities.items():
                kind = entity["kind"]
                _, entity_id = uid.rsplit(UID_SEP, 1)
                topic = "" if entity_id == "state" else entity_id + "_"
                if kind in ("State", "Transducer"):
                    msg[f'{topic}state_topic'] = f"leaf/state/{uid}"
                if kind in ("Actuator", "Transducer"):
                    msg[f'{topic}command_topic'] = f"leaf/act/{uid}"
            # print("DISCOVER", f"homeassistant/{domain}/{unique_id}/config", json.dumps(msg, indent=2))
            await client.publish(f"homeassistant/{domain}/{unique_id}/config", json.dumps(msg))

    async def action_listener(self):
        # forward action events from mqtt to eventbus
        await self.client.subscribe("leaf/act/#")
        async for msg in self.client.messages:
            uid = str(msg.topic).split("/")[-1]
            print("EMIT", {"topic": "?act", "uid": uid, "state": msg.payload, "src": "#hass"})
            await bus.emit({"topic": "?act", "uid": uid, "state": msg.payload, "src": "#hass"})



async with aiomqtt.Client(URL) as client:

    hass_mqtt = HassMqtt(client)

    await bus.emit_kwargs(topic="?device")

    await bus.emit_kwargs(topic='?act', uid=light1.uid('state'), value='on', dst='leaf_id')
    await bus.emit_kwargs(topic='?act', uid=light1.uid('brightness'), value=55)

    for i in range(60000): await asyncio.sleep(0.001)

print("DONE")

